In [1]:
!pip install fairseq2
!pip install pydub sentencepiece 
!pip install git+https://github.com/facebookresearch/seamless_communication.git
!pip install demucs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.8/513.8 kB 12.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 5.7 MB/s eta 0:00:000:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/6

In [2]:
import io
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import mmap
import numpy
import soundfile
import torchaudio
import torch

from collections import defaultdict
from IPython.display import Audio, display
from pathlib import Path
from pydub import AudioSegment

from seamless_communication.inference import Translator
from seamless_communication.streaming.dataloaders.s2tt import SileroVADSilenceRemover

In [3]:
!wget "https://d11ywzt2xtszji.cloudfront.net/SeamlessExpressive.tar.gz?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoicG1rM3YxN2U4Zmo0bHRqMW11dXJrZmw5IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZDExeXd6dDJ4dHN6amkuY2xvdWRmcm9udC5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NTk5MDQ2MjB9fX1dfQ__&Signature=WTIoYAT5G6JnLyLb1%7EjHA1sYjY10hySNDdXRQcjVTuR1M1bUcAez6xoB%7EsTCWi6V-cO7nGeIbRq14Vb0HuMY1zl17wQFjWdqdlSB8fdoud161LEv3lLOM8eYwAqv3nrjAGTJRcI8Ya0-DhYw7jSL6gTjDZZK6qlIgxsqzzgPYdsyRqdSi6ay9-iyp2rszw7Eyd-8Ys5u2DZATkQF5FvARWpFapYUdbmMvh-8Sy%7Ez1cXp97jl7mZQwUnvbKyFxtWV1wQ801iAzNrMWjJ1SjzogX%7EHQ02zYoQHmKO-JFPV9qyCnoOnn0po%7EMESWaWLtneUBIG8SWTKHVqxbig1%7EyAV9A__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1335365918247659" -O /content/SeamlessExpressive.tar.gz

!tar -xzvf /content/SeamlessExpressive.tar.gz

--2025-10-06 07:51:47--  https://d11ywzt2xtszji.cloudfront.net/SeamlessExpressive.tar.gz?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoicG1rM3YxN2U4Zmo0bHRqMW11dXJrZmw5IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZDExeXd6dDJ4dHN6amkuY2xvdWRmcm9udC5uZXRcLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3NTk5MDQ2MjB9fX1dfQ__&Signature=WTIoYAT5G6JnLyLb1%7EjHA1sYjY10hySNDdXRQcjVTuR1M1bUcAez6xoB%7EsTCWi6V-cO7nGeIbRq14Vb0HuMY1zl17wQFjWdqdlSB8fdoud161LEv3lLOM8eYwAqv3nrjAGTJRcI8Ya0-DhYw7jSL6gTjDZZK6qlIgxsqzzgPYdsyRqdSi6ay9-iyp2rszw7Eyd-8Ys5u2DZATkQF5FvARWpFapYUdbmMvh-8Sy%7Ez1cXp97jl7mZQwUnvbKyFxtWV1wQ801iAzNrMWjJ1SjzogX%7EHQ02zYoQHmKO-JFPV9qyCnoOnn0po%7EMESWaWLtneUBIG8SWTKHVqxbig1%7EyAV9A__&Key-Pair-Id=K15QRJLYKIFSLZ&Download-Request-ID=1335365918247659
Resolving d11ywzt2xtszji.cloudfront.net (d11ywzt2xtszji.cloudfront.net)... 54.192.55.12, 54.192.55.84, 54.192.55.83, ...
Connecting to d11ywzt2xtszji.cloudfront.net (d11ywzt2xtszji.cloudfront.net)|54.192.55.12|:443... connected.
HTTP reques

In [4]:
import os

# Paths
input_video = "/kaggle/input/sample-video/input_video.mp4"  # Change to your actual video file path
output_dir = "/kaggle/working/OUTPUT"
os.makedirs(output_dir, exist_ok=True)

output_audio = os.path.join(output_dir, "output_audio.wav")
output_video = os.path.join(output_dir, "output_video.mp4")

# Extract audio as WAV
!ffmpeg -i "{input_video}" -vn -ac 2 -ar 44100 -c:a pcm_s16le "{output_audio}" -y

# Extract video (with original audio or without depending on your need)
# Here, extracting video with original audio intact
!ffmpeg -i "{input_video}" -c copy -an "{output_video}" -y

print("Separation completed!")
print("Audio file:", output_audio)
print("Video file:", output_video)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [5]:
import os
from pathlib import Path

# Create the output directory
output_dir = "/kaggle/working/DEMUCS"
os.makedirs(output_dir, exist_ok=True)

# Input audio file path
input_audio = "/kaggle/working/OUTPUT/output_audio.wav"  # Change this to your actual input path

# Run demucs and specify the output folder
!demucs --out={output_dir} {input_audio}

print("Demucs separation completed! Outputs are in", output_dir)


Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|███████████████████████████████████████| 80.2M/80.2M [00:00<00:00, 164MB/s]
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /kaggle/working/DEMUCS/htdemucs
Separating track /kaggle/working/OUTPUT/output_audio.wav
100%|██████████████████████████████████████████████████████████████████████████| 58.5/58.5 [00:08<00:00,  6.96seconds/s]
Demucs separation completed! Outputs are in /kaggle/working/DEMUCS


In [ ]:
!pip install resemble-enhance --upgrade

In [ ]:
!resemble-enhance /kaggle/working/DEMUCS/htdemucs/output_audio /kaggle/working

In [ ]:
# import os
# import torch
# import torchaudio

# # Load Silero VAD
# model, utils = torch.hub.load(
#     repo_or_dir='snakers4/silero-vad',
#     model='silero_vad',
#     force_reload=True,
#     trust_repo=True
# )
# (get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils

# audio_path = "/kaggle/working/vocals.wav"

# # Create output folder
# vad_folder = "/kaggle/working/VAD"
# os.makedirs(vad_folder, exist_ok=True)

# # Load audio
# waveform, sr = torchaudio.load(audio_path)

# # Convert to mono if stereo
# if waveform.shape[0] > 1:
#     waveform = waveform.mean(dim=0, keepdim=True)

# # Resample to 16kHz
# target_sr = 16000
# if sr != target_sr:
#     resampler = torchaudio.transforms.Resample(sr, target_sr)
#     waveform = resampler(waveform)
#     sr = target_sr

# # Run VAD
# speech_timestamps = get_speech_timestamps(waveform.squeeze(), model, sampling_rate=sr)

# print("Detected speech segments:")
# for seg in speech_timestamps:
#     print(f"Start: {seg['start']/sr:.2f}s, End: {seg['end']/sr:.2f}s")

# # Save each detected speech segment into VAD folder
# for idx, seg in enumerate(speech_timestamps):
#     chunk = waveform[:, seg['start']:seg['end']]  # shape: [1, samples]

#     # Ensure it's always [channels, samples]
#     if chunk.dim() == 1:
#         chunk = chunk.unsqueeze(0)

#     out_path = os.path.join(vad_folder, f"speech_segment_{idx}.wav")
#     torchaudio.save(out_path, chunk, sample_rate=sr)
#     print(f"Saved segment {idx} -> {out_path}")


In [6]:
import os
import json
import torch
import torchaudio
from pydub import AudioSegment, silence

# --------------------------
# Function to split long segments
# --------------------------
def split_if_too_long(audio_path, start_ms, end_ms, max_len=10, min_silence_len=200, silence_thresh=-40):
    """
    Split a segment only if it's longer than max_len (in seconds).
    Otherwise, keep it as is.
    Returns list of (start_ms, end_ms).
    """
    audio = AudioSegment.from_file(audio_path)
    segment_audio = audio[start_ms:end_ms]
    duration = len(segment_audio)

    if duration <= max_len * 1000:
        return [(start_ms, end_ms)]

    # Try to find silence near midpoint
    midpoint = duration // 2
    search_window = segment_audio[max(0, midpoint - 3000):min(duration, midpoint + 3000)]

    silences = silence.detect_silence(
        search_window,
        min_silence_len=min_silence_len,
        silence_thresh=silence_thresh
    )

    if silences:
        # Pick silence closest to midpoint
        silence_mid = min(silences, key=lambda s: abs((s[0]+s[1])//2 - midpoint))
        cut_point = max(0, midpoint - 3000) + (silence_mid[0] + silence_mid[1]) // 2
    else:
        # If no silence, just hard cut at midpoint
        cut_point = midpoint

    # Return two chunks (recursively split further if needed)
    left = split_if_too_long(audio_path, start_ms, start_ms + cut_point, max_len, min_silence_len, silence_thresh)
    right = split_if_too_long(audio_path, start_ms + cut_point, end_ms, max_len, min_silence_len, silence_thresh)
    return left + right


# --------------------------
# Main VAD pipeline
# --------------------------
# Load Silero VAD
model, utils = torch.hub.load(
    repo_or_dir='snakers4/silero-vad',
    model='silero_vad',
    force_reload=True,
    trust_repo=True
)
(get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils

audio_path = "/kaggle/working/DEMUCS/htdemucs/output_audio/vocals.wav"

# Create output folder
vad_folder = "/kaggle/working/VAD"
os.makedirs(vad_folder, exist_ok=True)

# Load audio
waveform, sr = torchaudio.load(audio_path)

# Convert to mono if stereo
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

# Resample to 16kHz
target_sr = 16000
if sr != target_sr:
    resampler = torchaudio.transforms.Resample(sr, target_sr)
    waveform = resampler(waveform)
    sr = target_sr

# Run VAD
speech_timestamps = get_speech_timestamps(waveform.squeeze(), model, sampling_rate=sr)

print("Detected speech segments (before splitting):")
for seg in speech_timestamps:
    print(f"Start: {seg['start']/sr:.2f}s, End: {seg['end']/sr:.2f}s")

# Save final segments (after splitting) + JSON
all_segments = []
segment_counter = 0
audio_full = AudioSegment.from_file(audio_path)

for seg in speech_timestamps:
    start_ms = int(seg['start'] / sr * 1000)
    end_ms = int(seg['end'] / sr * 1000)

    # Split if too long
    sub_segments = split_if_too_long(audio_path, start_ms, end_ms, max_len=10)

    for (s, e) in sub_segments:
        chunk = audio_full[s:e]
        out_path = os.path.join(vad_folder, f"speech_segment_{segment_counter}.wav")
        chunk.export(out_path, format="wav")

        all_segments.append({
            "id": segment_counter,
            "file": out_path,
            "start": s / 1000.0,
            "end": e / 1000.0
        })

        print(f"💾 Saved segment {segment_counter} -> {out_path} ({(e-s)/1000:.2f}s)")
        segment_counter += 1

# Save JSON metadata
json_path = os.path.join(vad_folder, "segments.json")
with open(json_path, "w") as f:
    json.dump(all_segments, f, indent=4)

print(f"\n📑 JSON saved -> {json_path}")


Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


Detected speech segments (before splitting):
Start: 0.00s, End: 9.73s
Start: 9.83s, End: 16.19s
Start: 16.29s, End: 23.07s
Start: 23.30s, End: 23.65s
Start: 23.94s, End: 29.05s
Start: 29.41s, End: 31.36s
Start: 31.87s, End: 35.68s
Start: 35.81s, End: 36.38s
Start: 36.80s, End: 48.19s
Start: 48.35s, End: 52.36s
💾 Saved segment 0 -> /kaggle/working/VAD/speech_segment_0.wav (9.73s)
💾 Saved segment 1 -> /kaggle/working/VAD/speech_segment_1.wav (6.36s)
💾 Saved segment 2 -> /kaggle/working/VAD/speech_segment_2.wav (6.78s)
💾 Saved segment 3 -> /kaggle/working/VAD/speech_segment_3.wav (0.35s)
💾 Saved segment 4 -> /kaggle/working/VAD/speech_segment_4.wav (5.12s)
💾 Saved segment 5 -> /kaggle/working/VAD/speech_segment_5.wav (1.95s)
💾 Saved segment 6 -> /kaggle/working/VAD/speech_segment_6.wav (3.80s)
💾 Saved segment 7 -> /kaggle/working/VAD/speech_segment_7.wav (0.57s)
💾 Saved segment 8 -> /kaggle/working/VAD/speech_segment_8.wav (8.51s)
💾 Saved segment 9 -> /kaggle/working/VAD/speech_segment_9.

In [7]:
import os

# Input and output directories
vad_dir = "/kaggle/working/VAD"
output_dir = "/kaggle/working/translated"
os.makedirs(output_dir, exist_ok=True)

# Target language
tgt_lang = "spa"

# Loop through all VAD segments
for file in sorted(os.listdir(vad_dir)):
    if file.endswith(".wav"):
        in_file = os.path.join(vad_dir, file)
        out_file = os.path.join(output_dir, file)  # same name as input

        print(f"🔹 Translating {file} -> {out_file}")

        # Run Seamless Expressive translation
        !expressivity_predict {in_file} --tgt_lang {tgt_lang} \
            --model_name seamless_expressivity --vocoder_name vocoder_pretssel \
            --gated-model-dir SeamlessExpressive --output_path {out_file}

print("\n✅ All translated files saved in:", output_dir)


🔹 Translating speech_segment_0.wav -> /kaggle/working/translated/speech_segment_0.wav
100%|█████████████████████████████████████████| 360k/360k [00:00<00:00, 950kB/s]
100%|██████████████████████████████████████| 4.93M/4.93M [00:00<00:00, 65.8MB/s]
Using the cached tokenizer of seamless_expressivity. Set `force` to `True` to download again.
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
🔹 Translating speech_segment_1.wav -> /kaggle/working/translated/speech_segment_1.wav
Using the cached tokenizer of seamless_expressivity. Set `force` to `True` to download again.
Using the cached tokenizer of seamless_expressivity. Set `force` to `True` to download again.
Using the cached tokenizer of seamless_expressivity. Set `force` to `True` to down

In [10]:
import json
import os
from pydub import AudioSegment

# ========= CONFIG =========
vad_json = "/kaggle/working/VAD/segments.json"   # JSON with VAD timings
vad_folder = "/kaggle/working/VAD"                   # Original VAD segments folder
translated_folder = "/kaggle/working/translated"     # Translated audio segments folder
output_path = "/kaggle/working/final_aligned.wav"    # Final output

# ========= LOAD VAD TIMESTAMPS =========
with open(vad_json, "r") as f:
    vad_segments = json.load(f)

final_audio = AudioSegment.silent(duration=0)

for seg in vad_segments:
    idx = seg["id"]  # assuming JSON has {"id": 0, "start": .., "end": ..}
    start_ms = int(seg["start"] * 1000)
    end_ms = int(seg["end"] * 1000)
    duration_ms = end_ms - start_ms

    translated_path = os.path.join(translated_folder, f"speech_segment_{idx}.wav")
    if not os.path.exists(translated_path):
        print(f"⚠️ Missing translated file for seg {idx}, skipping...")
        continue

    seg_audio = AudioSegment.from_file(translated_path).set_channels(1).set_frame_rate(16000)

    # ===== Match duration =====
    if len(seg_audio) > duration_ms:
        seg_audio = seg_audio[:duration_ms]  # trim
        print(f"✂️ Trimmed seg {idx} to {duration_ms/1000:.2f}s")
    elif len(seg_audio) < duration_ms:
        silence_padding = AudioSegment.silent(duration=duration_ms - len(seg_audio))
        seg_audio = seg_audio + silence_padding
        print(f"➕ Padded seg {idx} to {duration_ms/1000:.2f}s")

    # ===== Insert gap if needed =====
    current_length = len(final_audio)
    if current_length < start_ms:
        gap = AudioSegment.silent(duration=start_ms - current_length)
        final_audio += gap
        print(f"🕐 Inserted silence before seg {idx}: {(start_ms - current_length)/1000:.2f}s")

    # ===== Append aligned segment =====
    final_audio += seg_audio
    print(f"✅ Added seg {idx} ({duration_ms/1000:.2f}s)")

# ========= EXPORT =========
final_audio.export(output_path, format="wav")
print(f"\n🎉 Final aligned audio saved at: {output_path}")


➕ Padded seg 0 to 9.73s
✅ Added seg 0 (9.73s)
➕ Padded seg 1 to 6.36s
🕐 Inserted silence before seg 1: 0.10s
✅ Added seg 1 (6.36s)
➕ Padded seg 2 to 6.78s
🕐 Inserted silence before seg 2: 0.10s
✅ Added seg 2 (6.78s)
✂️ Trimmed seg 3 to 0.35s
🕐 Inserted silence before seg 3: 0.23s
✅ Added seg 3 (0.35s)
✂️ Trimmed seg 4 to 5.12s
🕐 Inserted silence before seg 4: 0.29s
✅ Added seg 4 (5.12s)
✂️ Trimmed seg 5 to 1.95s
🕐 Inserted silence before seg 5: 0.36s
✅ Added seg 5 (1.95s)
➕ Padded seg 6 to 3.80s
🕐 Inserted silence before seg 6: 0.52s
✅ Added seg 6 (3.80s)
✂️ Trimmed seg 7 to 0.57s
🕐 Inserted silence before seg 7: 0.13s
✅ Added seg 7 (0.57s)
➕ Padded seg 8 to 8.51s
🕐 Inserted silence before seg 8: 0.42s
✅ Added seg 8 (8.51s)
➕ Padded seg 9 to 2.88s
✅ Added seg 9 (2.88s)
✂️ Trimmed seg 10 to 4.00s
🕐 Inserted silence before seg 10: 0.16s
✅ Added seg 10 (4.00s)

🎉 Final aligned audio saved at: /kaggle/working/final_aligned.wav


In [11]:
!pip install openvoice-cli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 5.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 44.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 74.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.7/801.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.7 MB/s eta 0:00:00
  

In [12]:
!python -m openvoice_cli single \
  -i /kaggle/working/final_aligned.wav \
  -r /kaggle/working/OUTPUT/output_audio.wav \
  -o /kaggle/working/cloned_output.wav \
  -d cuda

[OpenVoice Converter] Downloading checkpoint.pth...
100%|████████████████████████████████████████| 131M/131M [00:01<00:00, 103MiB/s]
[OpenVoice Converter] Downloading config.json...
100%|█████████████████████████████████████████| 838/838 [00:00<00:00, 2.86MiB/s]
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
step59000_snr39.99_pesq4.35_BERP_none0.3(…): 100%|█| 10.0M/10.0M [00:00<00:00, 1
Loaded checkpoint '/usr/local/lib/python3.11/dist-packages/openvoice_cli/checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []
OpenVoice version: v2
[(0.0, 8.818), (9.326, 15.538), (15.822, 22.322), (22.862, 35.282), (35.47, 52.3545)]
after vad: dur = 50.834
/usr/local/lib/python3.11/dist-packages/torch/functional.py:660: UserWarning: st

In [13]:
!git clone https://github.com/justinjohn0306/Wav2Lip.git


Cloning into 'Wav2Lip'...
remote: Enumerating objects: 534, done.
remote: Total 534 (delta 0), reused 0 (delta 0), pack-reused 534 (from 1)
Receiving objects: 100% (534/534), 29.78 MiB | 37.98 MiB/s, done.
Resolving deltas: 100% (272/272), done.


In [20]:
%cd Wav2Lip

/kaggle/working/Wav2Lip


In [17]:
!pip install numpy>=1.25 scipy>=1.12 torch>=2.0 torchvision>=0.15 opencv-python>=4.10 tqdm numba ffmpeg-python


In [21]:
# Step 2: Download pretrained model (standard Wav2Lip)
!mkdir -p checkpoints
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'


--2025-10-06 08:19:54--  https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/615543729/76281b9f-48b8-4cbf-9a05-edf61d847109?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-06T09%3A09%3A11Z&rscd=attachment%3B+filename%3Dwav2lip_gan.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-06T08%3A08%3A45Z&ske=2025-10-06T09%3A09%3A11Z&sks=b&skv=2018-11-09&sig=2AiPmQ50V43TwApbzYBPxxQ9Tq0zuuHwI7m0hzBfrfc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1OTc0MjM5NSwibmJmIjoxNzU5NzM4Nzk1LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5

In [22]:
# Step 3: Define your paths (your project files)
VIDEO_PATH = "/kaggle/working/OUTPUT/output_video.mp4"
TRANSLATED_AUDIO_PATH = "/kaggle/working/cloned_output.wav"
OUTPUT_VIDEO_PATH = "/kaggle/working/final_lip_synced_video.mp4"

In [23]:
!pip install batch-face==1.5.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 44.3 MB/s eta 0:00:0000:0100:01m


In [24]:
!wget https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth -O checkpoints/mobilenet.pth


--2025-10-06 08:20:56--  https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/615543729/0f1702ef-4998-4acd-abc8-b80c52e838b9?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-06T09%3A10%3A40Z&rscd=attachment%3B+filename%3Dmobilenet.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-06T08%3A10%3A15Z&ske=2025-10-06T09%3A10%3A40Z&sks=b&skv=2018-11-09&sig=vJdP1fUo10sopjXV6%2BaOJBtW2zxwty%2B7qAQozh8eIZU%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1OTczOTE1NywibmJmIjoxNzU5NzM4ODU3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5

In [25]:
!python inference.py \
    --checkpoint_path checkpoints/wav2lip_gan.pth \
    --face $VIDEO_PATH \
    --audio $TRANSLATED_AUDIO_PATH \
    --outfile $OUTPUT_VIDEO_PATH \
    --nosmooth


Using cuda for inference.
Load checkpoint from: checkpoints/wav2lip_gan.pth
Models loaded
Reading video frames...
Number of frames available for inference: 1569
(80, 4188)
Length of mel chunks: 1565
100%|███████████████████████████████████████████| 13/13 [00:24<00:00,  1.91s/it]
wav2lip prediction time: 24.855103969573975
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa 